<a href="https://colab.research.google.com/github/vinilvadakkepurakkal/chainbreakers/blob/master/COVIbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#self learning chatbot 
#Author : Vinil Vadakkepurakkal

In [227]:

!pip install  newspaper3k
!pip install PyPDF2

In [0]:
#import libraries
import nltk
from newspaper import Article
import random
import string
import numpy as np
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import io
import requests
from PyPDF2 import PdfFileReader

In [0]:
#ignore warning messages
warnings.filterwarnings("ignore")

In [231]:
#download the packages from NLTK
nltk.download('punkt',quiet=True)
nltk.download('wordnet', quiet=True)

True

In [0]:
#get the article
#article = Article('https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521')
#article = Article('https://www.who.int/health-topics/coronavirus#tab=tab_1')
article = Article('https://en.wikipedia.org/wiki/Coronavirus_disease_2019')
article.download()
article.parse()
article.nlp()
corpus = article.text
## URL from getting details
#url = 'https://www.who.int/docs/default-source/sri-lanka-documents/what-is-coronavirus-english.pdf?sfvrsn=a6b21ac_2'
#r = requests.get(url)
##f = io.BytesIO(r.content)
#reader = PdfFileReader(f)
#for i in range(0, reader.getNumPages()):
#   corpus += reader.getPage(i).extractText()

In [233]:
print(corpus)

Viral respiratory disease first detected in December 2019

Coronavirus disease 2019 (COVID-19) is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2).[8] The disease was first identified in December 2019 in Wuhan, the capital of China's Hubei province, and has since spread globally, resulting in the ongoing 2019–20 coronavirus pandemic.[9][10] As of 2 May 2020, more than 3.34 million cases have been reported across 187 countries and territories, resulting in more than 238,000 deaths. More than 1.05 million people have recovered.[7]

Common symptoms include fever, cough, fatigue, shortness of breath, and both loss of smell and taste.[11][5][12][13] While the majority of cases result in mild symptoms, some progress to viral pneumonia, multi-organ failure, or cytokine storm.[9][14][15] The time from exposure to onset of symptoms is typically around five days but may range from two to fourteen days.[5][16]

The virus is primarily spread between peop

In [234]:
#tokenization
text = corpus
sent_tokens = nltk.sent_tokenize(text) #converting text into a list of sentances
print(sent_tokens)


['Viral respiratory disease first detected in December 2019\n\nCoronavirus disease 2019 (COVID-19) is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2).', "[8] The disease was first identified in December 2019 in Wuhan, the capital of China's Hubei province, and has since spread globally, resulting in the ongoing 2019–20 coronavirus pandemic.", '[9][10] As of 2 May 2020, more than 3.34 million cases have been reported across 187 countries and territories, resulting in more than 238,000 deaths.', 'More than 1.05 million people have recovered.', '[7]\n\nCommon symptoms include fever, cough, fatigue, shortness of breath, and both loss of smell and taste.', '[11][5][12][13] While the majority of cases result in mild symptoms, some progress to viral pneumonia, multi-organ failure, or cytokine storm.', '[9][14][15] The time from exposure to onset of symptoms is typically around five days but may range from two to fourteen days.', '[5][16]\n\nThe vir

In [235]:
#creating a dictionary (key:value) pair to remove the punctations
remove_punct_dict = dict( (punct,None) for punct in string.punctuation)
remove_punct_dict = dict( (ord(punct),None) for punct in string.punctuation) #to get the numbers for punctuation

print(string.punctuation)
print(remove_punct_dict)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [236]:
#create a function to return a lemmitized lower case words after removing punctuation
def LemNormalize(text):
    return nltk.word_tokenize(text.lower().translate(remove_punct_dict))

print(LemNormalize(text))

['viral', 'respiratory', 'disease', 'first', 'detected', 'in', 'december', '2019', 'coronavirus', 'disease', '2019', 'covid19', 'is', 'an', 'infectious', 'disease', 'caused', 'by', 'severe', 'acute', 'respiratory', 'syndrome', 'coronavirus', '2', 'sarscov28', 'the', 'disease', 'was', 'first', 'identified', 'in', 'december', '2019', 'in', 'wuhan', 'the', 'capital', 'of', 'chinas', 'hubei', 'province', 'and', 'has', 'since', 'spread', 'globally', 'resulting', 'in', 'the', 'ongoing', '2019–20', 'coronavirus', 'pandemic910', 'as', 'of', '2', 'may', '2020', 'more', 'than', '334', 'million', 'cases', 'have', 'been', 'reported', 'across', '187', 'countries', 'and', 'territories', 'resulting', 'in', 'more', 'than', '238000', 'deaths', 'more', 'than', '105', 'million', 'people', 'have', 'recovered7', 'common', 'symptoms', 'include', 'fever', 'cough', 'fatigue', 'shortness', 'of', 'breath', 'and', 'both', 'loss', 'of', 'smell', 'and', 'taste1151213', 'while', 'the', 'majority', 'of', 'cases', 'r

In [0]:
#key word matching
# Greetings inputs
GREETING_INPUTS = ["hi", "hello", "hola", "greetings", "wassup", "hey"]
# Greeting responses
GREETING_RESPONSES = ["howdy","hi", "hey", "what's good", "hello", "hey there"]

#function to return a random response
def greeting(sentance):
    #if user input a greeting it should respond with a random greeting response
    for word in sentance.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)


In [238]:
print(greeting("hey"))

howdy


In [0]:
def response (user_response):
    # User response / query
    #user_response = "What is chronic kidney disease"
    user_response = user_response.lower()
    #### Print user reponse

    #print(user_response)
    #chatbot response
    robo_response = ''

    #append the user response to a sentance list
    sent_tokens.append(user_response)
    #### print the sentance list after appending the user response

    #print(sent_tokens)

    #create a TfidfVectorizer
    TfidfVec = TfidfVectorizer(tokenizer= LemNormalize ,stop_words='english')

    #convert text to a matrix of TF-IDF features
    tfidf = TfidfVec.fit_transform(sent_tokens)

    #### print tfidf features
    #print(tfidf)

    #get measures of the similarity (similarity scores) 
    vals = cosine_similarity(tfidf[-1], tfidf) # User response is at the end of the list ie.[-1].

    #### print similarity scores
    #print(vals)

    # get the index of most similar sentance/text to the user response
    idx = vals.argsort()[0][-2] # exclude the user response ie [-1], using [-2] to take the next one

    #reduce the dimentionality of vals ( vals is a list of list and now making to one list)
    flat = vals.flatten()

    #sort the list in acending order
    flat.sort()

    #get the most similar score for user response
    score = flat[-2]

    #### print the similarity score
    #print(score)

    #if the variable score is 0 then there is no text similar to user response

    if (score == 0) :
        robo_response = robo_response+"I apologize, I dont understand."
    else:
        robo_response = robo_response+sent_tokens[idx]

    # print the chatbot response
    #print(robo_response)

    # remove user response from sentance tokens

    sent_tokens.remove(user_response)
    return robo_response

In [0]:
#sent_tokens.remove(user_response)

In [241]:
flag = True
print("COVIBot : I am COVIBot (COVID-19 Bot). I will answer your queuries about COVID-19. If you want to exit type 'bye")
while(flag == True):
    user_response = input()
    user_response = user_response.lower()
    if (user_response != 'bye'):
        if (user_response == 'thanks' or user_response == 'thank you'):
            flag = False
            print("COVIBot : You are welcome!")
        else:
            if (greeting(user_response) != None):
                print("COVIBot : " +greeting(user_response))
            else:
                print("COVIBot : " +response(user_response))
                
    else:
        flag = False
        print("COVIBot : Chat with you later !")

COVIBot : I am COVIBot (COVID-19 Bot). I will answer your queuries about COVID-19. If you want to exit type 'bye
Hello 
COVIBot : hey
wearing masks?
COVIBot : [165] When masks are not available, the CDC recommends using face shields or, as a last resort, homemade masks.
wearing mask will help ?
COVIBot : [135][136][137] The CDC recommends that those who suspect they carry the virus wear a simple face mask.
 bye
COVIBot : I apologize, I dont understand.
bye
COVIBot : Chat with you later !
